In [2]:
import Pkg; 
Pkg.add("LinearMaps")
Pkg.add("IterativeSolvers")
Pkg.add("ProgressBars")

   Resolving package versions...
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
   Installed ProgressBars ─ v1.5.0
    Updating `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  [49802e3a] + ProgressBars v1.5.0
    Updating `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`
  [49802e3a] + ProgressBars v1.5.0
Precompiling project...
  ✓ ProgressBars
  1 dependency successfully precompiled in 1 seconds. 94 already precompiled.


In [2]:
using LinearMaps
using IterativeSolvers
using FFTW
using Base.Threads
using PaddedViews
using BenchmarkTools
using ChainRulesCore
using Zygote
using LinearAlgebra
using FiniteDifferences
using ProgressBars

function g2g̃(g)
    g̃ = complex.(zeros(size(g)),zeros(size(g)))
    n = size(g)[3]
    Threads.@threads for i in 1:n
        g̃[:,:,i] .= fft(g[:,:,i])
    end
    g̃
end

function g2g̃ᵀ(g)
    g̃ᵀ = complex.(zeros(size(g)),zeros(size(g)))
    n = size(g)[3]
    Threads.@threads for i in 1:n
        g̃ᵀ[:,:,i] .= fft(reverse(g[:,:,i]))
    end
    g̃ᵀ
end

⊛(g̃,u) = real.(ifftshift( ifft( g̃ .* fft(u) ) ))
✪(v,u) = real.(ifftshift( ifft( fft(v) .* conj.(fft(u)))))
syz(u,g̃) = collect(sym_paddedviews(0,u,g̃)[1])
v∂gu(g̃,u,v) = ✪(syz(v,g̃),syz(u,g̃))

function conv(u,K̃,fwd::Bool)
    nᵢ, nₖ = size(u)[1], size(K̃)[1]
    nₒ = nₖ-nᵢ

    i₁ = fwd==true ? 1+(nᵢ÷2) : nᵢ÷2
    i₂ = i₁+nₒ-1
    ⊛(K̃,syz(u,K̃))[i₁:i₂,i₁:i₂]
end

function G(u,g̃)
    nᵢ, nₖ = size(u)[1], size(g̃)[1]
    nₒ = nₖ-nᵢ
    n = size(g̃)[3]
    Gu = zeros(Float64,nₒ,nₒ,n)
    Threads.@threads for i in 1:n
        Gu[:,:,i] .= conv(u[:,:,i],g̃[:,:,i],true)
    end

    sum(Gu,dims=3)[:,:,1]
end

function GᵀG(u,g̃,g̃ᵀ)
    
    Gu = G(u,g̃)
    n = size(g̃)[3]
    GᵀGu = zeros(size(u))
    Threads.@threads for i in 1:n
        GᵀGu[:,:,i] .= conv(Gu,g̃ᵀ[:,:,i],false)
    end

    GᵀGu
end

function y∂GᵀGx(g̃,x,y)

    Gx = G(x,g̃)
    Gy = G(y,g̃)

    n = size(g̃)[3]
    ∂g = zeros(size(g̃))
    Threads.@threads for i in 1:n
        ∂g[:,:,i] .= v∂gu(g̃[:,:,i],x[:,:,i],Gy) .+ v∂gu(g̃[:,:,i],y[:,:,i],Gx)
    end
    
    ∂g
end

GᵀGαI(x,g̃,g̃ᵀ,α) = GᵀG(x,g̃,g̃ᵀ) .+ α.*x

function regress(g̃,g̃ᵀ,α,y)

    dims = size(y)

    A(ξ) = vec(GᵀGαI(reshape(ξ,dims),g̃,g̃ᵀ,α))
    Â = LinearMap(A,size(vec(y))[1],issymmetric=true,isposdef=true)
    println("Entering CG ..."); flush(stdout)
    ret = reshape(cg(Â,vec(y),reltol=10^(-16)),dims)
    println("CG has finished."); flush(stdout)

    ret

end

function G2Û!(g,α,U,g̃,g̃ᵀ)

    g̃[:,:,:] = g2g̃(g)[:,:,:]
    g̃ᵀ[:,:,:] = g2g̃ᵀ(g)[:,:,:]
    
    Û = similar(U)
    Threads.@threads for i in 1:size(U)[4]
        println("Reconstruction stage: ground truth $i"); flush(stdout)
        y = GᵀG(U[:,:,:,i],g̃,g̃ᵀ)
        Û[:,:,:,i] .= regress(g̃,g̃ᵀ,α,y)[:,:,:]
    end
    Û

end

function Û2G!(∇ℓ,g̃,g̃ᵀ,α,U,Û)

    (n1,n2,n3) = size(g̃)
    n4 = size(U)[4]
    ∂g = zeros(Float64,n1,n2,n3,n4)
    Threads.@threads for i in 1:size(Û)[4]
        println("Backpropagation stage: ground truth $i"); flush(stdout)
        Λ = regress(g̃,g̃ᵀ,α,∇ℓ[:,:,:,i])
        ∂g[:,:,:,i] .= y∂GᵀGx(g̃,U[:,:,:,i].-Û[:,:,:,i],Λ)[:,:,:]
    end
    sum(∂g,dims=4)[:,:,:,1]

end

function ChainRulesCore.rrule(::typeof(G2Û!), g,α,U,g̃,g̃ᵀ)
    Û = G2Û!(g,α,U,g̃,g̃ᵀ)
    function G2Û!_pullback(∇ℓ)

        ∂g = @thunk(Û2G!(∇ℓ,g̃,g̃ᵀ,α,U,Û))
        NoTangent(), ∂g, ZeroTangent(), ZeroTangent(), ZeroTangent(), ZeroTangent()

    end
    Û, G2Û!_pullback
end

function G2ℓ(g,U,α)

    g̃ = Array{ComplexF64,3}(undef,size(g))
    g̃ᵀ = Array{ComplexF64,3}(undef,size(g))
    Û = G2Û!(g,α,U,g̃,g̃ᵀ)

    ntrain = size(U)[4]
    ret = 0.0
    for i in 1:ntrain
        ret += norm(U[:,:,:,i].-Û[:,:,:,i])/norm(U[:,:,:,i])
    end

    ret/ntrain  
end

ng = 600
nu = 120
n = 20
g = rand(ng,ng,n)
u = rand(nu,nu,n)
α = 100000.0
ntrain = 5
U = rand(nu,nu,n,ntrain)

gdat = @time Zygote.gradient(x->G2ℓ(x,U,α),g)[1];


Reconstruction stage: ground truth 3
Reconstruction stage: ground truth 1
Reconstruction stage: ground truth 5
Reconstruction stage: ground truth 4
Reconstruction stage: ground truth 2
Entering CG ...
Entering CG ...
Entering CG ...
Entering CG ...
Entering CG ...
CG has finished.
CG has finished.
CG has finished.
CG has finished.
CG has finished.
Backpropagation stage: ground truth 2
Backpropagation stage: ground truth 3
Backpropagation stage: ground truth 1
Backpropagation stage: ground truth 4
Backpropagation stage: ground truth 5
Entering CG ...
Entering CG ...
Entering CG ...
Entering CG ...
Entering CG ...
CG has finished.
CG has finished.
CG has finished.
CG has finished.
CG has finished.
184.931195 seconds (3.72 M allocations: 414.007 GiB, 30.24% gc time, 2.37% compilation time: 7% of which was recompilation)


In [10]:
function G2û!(g,α,u,g̃,g̃ᵀ)

    g̃[:,:,:] = g2g̃(g)[:,:,:]
    g̃ᵀ[:,:,:] = g2g̃ᵀ(g)[:,:,:]
    y = GᵀG(u,g̃,g̃ᵀ)
    û = regress(g̃,g̃ᵀ,α,y)

    û
end

function û2G!(∇ℓ,g̃,g̃ᵀ,α,u,û)

    Λ = regress(g̃,g̃ᵀ,α,∇ℓ)

    y∂GᵀGx(g̃,u.-û,Λ)

end

function ChainRulesCore.rrule(::typeof(G2û!), g,α,u,g̃,g̃ᵀ)
    û = G2û!(g,α,u,g̃,g̃ᵀ)
    function G2û!_pullback(∇ℓ)

        ∂û = @thunk(û2G!(∇ℓ,g̃,g̃ᵀ,α,u,û))
        NoTangent(), ∂û, ZeroTangent(), ZeroTangent(), ZeroTangent(), ZeroTangent()

    end
    û, G2û!_pullback
end

function G2ℓ!(g,U,α)

    g̃ = Array{ComplexF64,3}(undef,size(g))
    g̃ᵀ = Array{ComplexF64,3}(undef,size(g))

    ntrain = size(U)[4]
    ret = 0.0
    for i in 1:ntrain
        û =  G2û!(g,α,U[:,:,:,i],g̃,g̃ᵀ)
        ret += norm(U[:,:,:,i].-û)/norm(U[:,:,:,i])
    end

    ret/ntrain  
end

ng = 8
nu = 4
n = 2
g = rand(ng,ng,n)
u = rand(nu,nu,n)
α = 10.0
ntrain = 2
U = rand(nu,nu,n,ntrain)

gdat = Zygote.gradient(x->G2ℓ!(x,U,α),g)[1];
Δ = grad(central_fdm(5,1),x->G2ℓ!(x,U,α),g)[1];
display(maximum(abs.(Δ .- gdat))/mean(abs.(Δ)))

4.041212136722791e-11

In [2]:
# function yGᵀGx(x,g,y)
#     g̃ = g2g̃(g)
#     g̃ᵀ = g2g̃ᵀ(g)
#     sum(y.*GᵀG(x,g̃,g̃ᵀ))
# end

# ng = 6
# n = 1
# nu = 2
# nv = ng - nu
# g = rand(ng,ng,n)
# u = rand(nu,nu,n)
# v = rand(nv,nv,n)
# α = 10.0
# ntrain = 2
# U = rand(nu,nu,n,ntrain)

# x = rand(nu,nu,n)
# y = rand(nu,nu,n)
# g̃,g̃ᵀ = g2g̃(g), g2g̃ᵀ(g);
# gdat = y∂GᵀGx(g̃,x,y);
# Δ = grad(central_fdm(5,1),Γ->yGᵀGx(x,Γ,y),g)[1];
# display(maximum(abs.(Δ .- gdat))/mean(abs.(Δ)))
# a, b = rand(1), rand(1)
# tmp = GᵀGαI(a.*x + b.*y, g̃,g̃ᵀ,α) 
# tmp1 = a.*GᵀGαI(x,g̃,g̃ᵀ,α)
# tmp2 = b.*GᵀGαI(y,g̃,g̃ᵀ,α)
# display(maximum(abs.(tmp .- tmp1 .- tmp2)))

# dims = size(x)
# A(ξ) = vec(GᵀGαI(reshape(ξ,dims),g̃,g̃ᵀ,α))
# tmp = A(a.*vec(x) + b.*vec(y)) .- a.*A(vec(x)) - b.*A(vec(y))
# display(maximum(abs.(tmp)))

function regress(g̃,g̃ᵀ,α,y)

    dims = size(y)

    A(ξ) = vec(GᵀGαI(reshape(ξ,dims),g̃,g̃ᵀ,α))
    Â = LinearMap(A,size(vec(y))[1],issymmetric=true,isposdef=true)
    reshape(cg(Â,vec(y),reltol=10^(-16)),dims)

end

function G2û(g,α,u)

    g̃ = g2g̃(g)
    g̃ᵀ= g2g̃ᵀ(g)
    y = GᵀG(u,g̃,g̃ᵀ)
    û = regress(g̃,g̃ᵀ,α,y)

    û
end

function û2G(∇ℓ,g,α,u,û)

    g̃ = g2g̃(g)
    g̃ᵀ= g2g̃ᵀ(g)
    Λ = regress(g̃,g̃ᵀ,α,∇ℓ)

    y∂GᵀGx(g̃,u.-û,Λ)

end

function ChainRulesCore.rrule(::typeof(G2û), g,α,u)
    û = G2û(g,α,u)
    function G2û_pullback(∇ℓ)

        ∂û = @thunk(û2G(∇ℓ,g,α,u,û))
        NoTangent(), ∂û, ZeroTangent(), ZeroTangent()

    end
    û, G2û_pullback
end

function G2ℓ(g,α,u)

    û =  G2û(g,α,u)
    norm(u.-û)/norm(u)
  
end

ng = 8
nu = 4
n = 2
g = rand(ng,ng,n)
u = rand(nu,nu,n)
α = 10.0
gdat = Zygote.gradient(x->G2ℓ(x,α,u),g)[1];
Δ = grad(central_fdm(5,1),x->G2ℓ(x,α,u),g)[1];
display(maximum(abs.(Δ .- gdat))/mean(abs.(Δ)))


5.111066082286911e-11

In [2]:
function regress(g̃,g̃ᵀ,α,y)

    dims = size(y)

    A(ξ) = vec(GᵀGαI(reshape(ξ,dims),g̃,g̃ᵀ,α))
    Â = LinearMap(A,size(vec(y))[1],issymmetric=true,isposdef=true)
    reshape(cg(Â,vec(y),reltol=10^(-16)),dims)

end

function G2û!(g,α,u,g̃,g̃ᵀ)

    g̃[:,:,:] = g2g̃(g)[:,:,:]
    g̃ᵀ[:,:,:] = g2g̃ᵀ(g)[:,:,:]
    y = GᵀG(u,g̃,g̃ᵀ)
    û = regress(g̃,g̃ᵀ,α,y)

    û
end

function û2G!(∇ℓ,g̃,g̃ᵀ,α,u,û)

    Λ = regress(g̃,g̃ᵀ,α,∇ℓ)

    y∂GᵀGx(g̃,u.-û,Λ)

end

function ChainRulesCore.rrule(::typeof(G2û!), g,α,u,g̃,g̃ᵀ)
    û = G2û!(g,α,u,g̃,g̃ᵀ)
    function G2û!_pullback(∇ℓ)

        ∂û = @thunk(û2G!(∇ℓ,g̃,g̃ᵀ,α,u,û))
        NoTangent(), ∂û, ZeroTangent(), ZeroTangent(), ZeroTangent(), ZeroTangent()

    end
    û, G2û!_pullback
end

function G2ℓ!(g,α,u)

    g̃ = complex.(zeros(size(g)),zeros(size(g)))
    g̃ᵀ = complex.(zeros(size(g)),zeros(size(g)))
    û =  G2û!(g,α,u,g̃,g̃ᵀ)
    norm(u.-û)/norm(u)
  
end

ng = 8
nu = 4
n = 2
g = rand(ng,ng,n)
u = rand(nu,nu,n)
α = 10.0
gdat = Zygote.gradient(x->G2ℓ!(x,α,u),g)[1];
Δ = grad(central_fdm(5,1),x->G2ℓ!(x,α,u),g)[1];
display(maximum(abs.(Δ .- gdat))/mean(abs.(Δ)))

1.0503185845704046e-10

In [31]:
a = rand(3,3)
dims = size(a)
display(a)
a = vec(a)
display(a)
a = reshape(a,dims)

3×3 Matrix{Float64}:
 0.61014   0.142683  0.442251
 0.82297   0.61583   0.820144
 0.638666  0.148343  0.207111

9-element Vector{Float64}:
 0.6101398049155026
 0.8229698999016793
 0.6386662590420531
 0.14268287115596867
 0.6158302059999865
 0.14834304962597256
 0.44225118960331444
 0.8201439526840497
 0.20711123005888388

3×3 Matrix{Float64}:
 0.61014   0.142683  0.442251
 0.82297   0.61583   0.820144
 0.638666  0.148343  0.207111

In [50]:
rand(1)

1-element Vector{Float64}:
 0.3144782679242779